In [ ]:
from sage.matrix.berlekamp_massey import berlekamp_massey
from sage.arith.functions import LCM_list
import math
import random
import numpy as np
import time

def binary(n, length):
    n = Z(n)
    if length < n.nbits():
        raise ValueError(format(n,length))
    return '0' * (length-n.nbits()) + n.binary()


Z = IntegerRing()


def linear_recursion(S, alpha):
    degree = len(alpha)-1
    for i in range(len(S)-degree):
        RHS = 0
        for j in range(degree):
            RHS = (RHS + S[j+i]*alpha[j])%2
        if (S[degree + i] != RHS):
            return false
    return true


def minimal_S(S):
    s = "GF(2)"
    s += f"({S[0]})"
    for i in range(1,len(S)):
        s += f",{int(S[i])}"
        
    s = f"berlekamp_massey([{s}])"
    m = eval(s)
    return m


def S_from_SFy(SFy, i, M):
    S = []
    for j in range(M):
        S.append(int(SFy[j][i]))
    return S  


def minimal_SFy(SFy, l):
    S_0 = S_from_SFy(SFy, 0, l**2)
    m = minimal_S(S_0)
    for i in range(1,l):
        if not (linear_recursion(S_from_SFy(SFy,i,l**2), m.list())):
            m_temp = minimal_S(S_from_SFy(SFy,i,l**2))
            R.<x> = PolynomialRing(GF(2))
            m = lcm(m, m_temp)
            

            return m 


def LocalInverse_SFy(SFy,l):
    x = 0
    m = minimal_SFy(SFy,l)
#     print(m)
    for i in range(l):
        x = x + sage.rings.integer.Integer(LocalInverse_S(S_from_SFy(SFy,i,l**2), m)) * (2**i)
    return x


def LocalInverse_S(S,m):
    degree = len(m.list()) - 1
    m_coeff = m.list()
    first_term = S[degree - 1]
    second_term = 0
    for i in (1..(degree-1)):
        second_term = (second_term + m_coeff[i]*S[i-1])%2
    
    localInverse_x = ((first_term - second_term)/m_coeff[0])%2
    return localInverse_x


def dec_to_bin(SFy, l):
    for i in range(len(SFy)):
        SFy[i] = binary(SFy[i],l)
        SFy[i] = SFy[i][::-1]
    return SFy


def func_e(e,n,SEy,l):
    for i in range (1,l*l):
        SEy.append(power_mod(SEy[i-1],e,n))
    
    return SEy


def func_d(c,n,SDy,l):
    for i in range (1,l*l):
        SDy.append(power_mod(c,SDy[i-1],n))
    
    return SDy



def P_Q_E(no_of_sets, bs_min, bs_max):
    primelist = []
    for i in range(2**bs_min - 1, 2**bs_max):
        if is_prime(i):
            primelist.append(i)
    primelist = prime_range(256,512)
    
    p_q = random.sample(primelist, k=2*no_of_sets)
    

    
    triplet_set = []
    for i in range(no_of_sets):
        phi_n = (p_q[2*i] - 1)*(p_q[2*i + 1] - 1)
        for j in range(2,phi_n):
            if (gcd(j, phi_n) == 1):
                triplet_set.append([p_q[2*i], p_q[2*i + 1], j])
                break
    return triplet_set
        

def LocalInverse_E_Map(sample_size, p, q, e):
    n = p*q
    c_list = random.sample(range(Z(1), Z(n-1)), k=sample_size)
    l = (int(math.log(n,2) + 2)//2)*2
    M = l*l
    y_x = []
    for i in range(sample_size):
        if(mod(i,1000) == 0):
            print(i)
        SEy = [c_list[i]]
        SEy_original = func_e(e,n,SEy,l)
        SEy = SEy_original.copy()
        SEy = dec_to_bin(SEy,l)
    
        try:
            LI_E = LocalInverse_SFy(SEy,l)

            if (SEy_original[0] == power_mod(LI_E,e,n)):
                y_x.append([c_list[i], LI_E])
                continue
            else:
                continue
        except:
            continue
    
    return y_x


def E_map_outcomes(sample_sizes, bs_min, bs_max):
    bit_length = bs_max
    table_parameters_E = []
    if (len(sample_sizes) == 1):  
        no_of_sets = sample_sizes[0][0]
        sample_size = sample_sizes[0][1]
        triplet_set = P_Q_E(no_of_sets, bs_min, bs_max)
        for i in range(no_of_sets):
            time_s = time.time()
            print(i+1, end=', ')
            table_parameters_E_temp = [triplet_set[i][0], triplet_set[i][1], triplet_set[i][2], triplet_set[i][0] * triplet_set[i][1], bit_length, sample_size]
            y_x = LocalInverse_E_Map(sample_size, triplet_set[i][0], triplet_set[i][1], triplet_set[i][2])
            density_E = float(len(y_x))/sample_size
            table_parameters_E_temp.append(density_E)
            time_e = time.time()
            table_parameters_E_temp.append(time_e - time_s)
            table_parameters_E.append(table_parameters_E_temp)
            print("\n\n","table_parameters_E", table_parameters_E_temp,"\n\n")
            
            
    else:
        no_of_sets = len(sample_sizes)
        triplet_set = P_Q_E(no_of_sets, bs_min, bs_max)
        for i in range(no_of_sets):
            print(i+1, end=', ')
            table_parameters_E_temp = [triplet_set[i][0], triplet_set[i][1], triplet_set[i][2], triplet_set[i][0] * triplet_set[i][1], bit_length, sample_sizes[i]]
            y_x = LocalInverse_E_Map(sample_sizes[i], triplet_set[i][0], triplet_set[i][1], triplet_set[i][2]) 
            density_E = float(len(y_x))/sample_sizes[i]
            table_parameters_E_temp.append(density_E)            
            table_parameters_E.append(table_parameters_E_temp)            

                
    print("\n\n","table_parameters_E", table_parameters_E,"\n\n")
    return None


def LocalInverse_D_Map(sample_size, p, q, e):
    n = p*q
    m_list = random.sample(range(Z(1), Z(n-1)), k=sample_size)
    l = (int(math.log(n,2) + 2)//2)*2
    M = l*l
    y_x = []
    for i in range(sample_size):
        if(mod(i,1000) == 0):
            print(i)
        SDy = [m_list[i]]
        SDy_original = func_d(power_mod(SDy[0], e, n),n,SDy,l)
        SDy = SDy_original.copy()
        SDy = dec_to_bin(SDy,l)
        try:
            LI_D = LocalInverse_SFy(SDy,l)
            c = power_mod(m_list[i],e,n)
            if (SDy_original[0] == power_mod(c,LI_D,n)):

                y_x.append([m_list[i], LI_D])
                print(sample_size, p, q, e, LI_D)
                continue
            else:
                continue
        except:
            continue
    
    return y_x


def D_map_outcomes(sample_sizes, bs_min, bs_max):
    bit_length = bs_max
    table_parameters_D = []
    if (len(sample_sizes) == 1):  
        no_of_sets = sample_sizes[0][0]
        sample_size = sample_sizes[0][1]
        triplet_set = P_Q_E(no_of_sets, bs_min, bs_max)
        for i in range(no_of_sets):
            time_s = time.time()
            print(i+1, end=', ')
            table_parameters_D_temp = [triplet_set[i][0], triplet_set[i][1], triplet_set[i][2], triplet_set[i][0] * triplet_set[i][1], bit_length, sample_size]
            y_x = LocalInverse_D_Map(sample_size, triplet_set[i][0], triplet_set[i][1], triplet_set[i][2])
            density_D = float(len(y_x))/sample_size
            table_parameters_D_temp.append(density_D)
            time_e = time.time()
            table_parameters_D_temp.append(time_e - time_s)
            table_parameters_D.append(table_parameters_D_temp)
            print("\n\n","table_parameters_D", table_parameters_D_temp,"\n\n")
            
            
    else:
        no_of_sets = len(sample_sizes)
        triplet_set = P_Q_E(no_of_sets, bs_min, bs_max)
        for i in range(no_of_sets):
            print(i+1, end=', ')
            table_parameters_D_temp = [triplet_set[i][0], triplet_set[i][1], triplet_set[i][2], triplet_set[i][0] * triplet_set[i][1], bit_length, sample_sizes[i]]
            y_x = LocalInverse_D_Map(sample_sizes[i], triplet_set[i][0], triplet_set[i][1], triplet_set[i][2]) 
            density_D = float(len(y_x))/sample_sizes[i]
            table_parameters_D_temp.append(density_D)
            table_parameters_D.append(table_parameters_D_temp)
            print("\n\n","table_parameters_D", table_parameters_D_temp,"\n\n")
                
    print("\n\n","table_parameters_D", table_parameters_D,"\n\n")
    return None


In [ ]:
sample_sizes = [[1, 10000]]
bs_min = 9
bs_max = 11
E_map_outcomes(sample_sizes, bs_min, bs_max)
time.sleep(1)
D_map_outcomes(sample_sizes, bs_min, bs_max)